In [1]:
import tensorflow_hub as hub
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import f1_score, accuracy_score

## Bert Classifier

### Load Data

In [2]:
#Open Data With Pickle
with open('Data.pkl', 'rb') as f:
   data = pickle.load(f)

In [4]:
data

,name,time,post,year,location,party,labor,geographical_area,negative,target,Elections,Tokens,bert,Vectors,tfidf
0,Rami_Greenberg,14/05/2018 23:13,בזמן שאני במילואים משרת את המדינה אני מקבל עדכ...,2018.0,Petah Tikva,הליכוד,Right,Center,True,NaN,True,"[זמן, במילואים, מדינה, עדכון, שטח, עבריין, נשל...","[0.16135516462782107, 0.17096072217797578, 0.6...","[[0.09727664, -0.034328427, -0.07360464, 0.004...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.08299487732492938,..."
1,Itai_Sahar,17/07/2018 21:31,ראיון ברשת ב' לגבי הזנחת מיגון השכונות הוותיקו...,2018.0,Ashkelon,"תקווה חדשה,מופת",Right,South,True,NaN,True,"[ראיון, ב', שכונה, התחממות, שבת, הפתיע, מוקד, ...","[0.12134942523386728, 0.21913829266070803, 0.6...","[[0.058681607, 0.0714004, -0.015508179, -0.024...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,Itai_Sahar,25/09/2018 13:51,החיבור בין הנהגה עירונית להנהגה רוחנית באשקלון...,2018.0,Ashkelon,"תקווה חדשה,מופת",Right,South,True,NaN,True,"[חיבור, הנהגה, הנהגה, גבול, תמיכה, תמיכה, דעת,...","[0.060134747054776415, 0.21581373673541834, 0....","[[0.08643902, 0.02041652, 0.049621165, 0.00082...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,Efraim Bulmash,17/09/2018 19:44,"אם אני הייתי ראש עיריית קריית גת, כבוד הרב היה...",2018.0,Netanya,העבודה,Left,Center,True,NaN,True,"[רב, רב, בלין,, התנצל, התפלל, כיפור, סליחה, עד...","[5.125870666233823e-05, 6.046074122423306e-05,...","[[-0.07948438, -0.053431872, 0.24393372, 0.028...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,Efraim Bulmash,26/06/2018 08:19,קאט דה בולשיט שאנז אליזה ברחוב הרצל! קודם תנ...,2018.0,Netanya,העבודה,Left,Center,True,NaN,True,"[רחוב, תנגישו, עיר, מערך, ייצר, חנייה, כלל, חנ...","[5.993927698000334e-05, 4.033872755826451e-05,...","[[-0.070441075, 0.025691396, 0.11220714, -0.04...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068,Shimon Keznelson,28/06/2018 14:53,"מוטי אלבז היקר תודה רבה על האירוח הלבבי, תודה ...",2018.0,Ashdod,אשדוד ביחד,NaN,South,False,NaN,True,"[תודה, אירוח, הודה, תושב, עיר, הגיע, הביע, תמי...","[0.0001437089085811749, 0.9997618794441223, 9....","[[-0.044175133, 0.08961652, 0.07179671, -0.008...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1069,Itzik Braverman,21/06/2018 09:46,הודעתי אתמול על הקמת יחידה לפיקוח על הגנים הפר...,2018.0,Petah Tikva,"אכפ""ת,העבודה",Left,Center,False,NaN,True,"[הודיע, יחידה, פיקוח, גן, ערב, התקיים, מפגש, פ...","[0.10708105028121888, 0.8066147193312645, 0.08...","[[0.105399854, 0.099687375, -0.0052871583, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1070,Raz Kinstlich,13/08/2018 08:45,"מעין פלח שדה, 50, בעלת תואר שני במנהל עסקים. ...",2018.0,Rishon LeZion,תקווה חדשה,Right,Center,False,NaN,True,"[תואר, מנהל, עסק, רכש, מפעל, הייטק, דירקטור, מ...","[0.31750033317366616, 0.676763616874814, 0.005...","[[0.14289406, 0.057123892, 0.05320154, 0.00059...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1071,Ifat Kariv,28/10/2018 19:05,לא מבזבזים את הקול! לתיבות הדואר שלכם הגיעה ...,2018.0,Hod Hasharon,NaN,NaN,Center,False,NaN,True,"[קול, דואר, הגיע, חובר, מידע, בחירה, מידע, תוכ...","[0.161508748215662, 0.3438450593712332, 0.4946...","[[0.18943638, -0.048461493, -0.11169901, -0.15...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


### Train Model

In [5]:
train_data = data[['post','negative']]

In [6]:
train_data

,post,negative
0,בזמן שאני במילואים משרת את המדינה אני מקבל עדכ...,True
1,ראיון ברשת ב' לגבי הזנחת מיגון השכונות הוותיקו...,True
2,החיבור בין הנהגה עירונית להנהגה רוחנית באשקלון...,True
3,"אם אני הייתי ראש עיריית קריית גת, כבוד הרב היה...",True
4,קאט דה בולשיט שאנז אליזה ברחוב הרצל! קודם תנ...,True
...,...,...
1068,"מוטי אלבז היקר תודה רבה על האירוח הלבבי, תודה ...",False
1069,הודעתי אתמול על הקמת יחידה לפיקוח על הגנים הפר...,False
1070,"מעין פלח שדה, 50, בעלת תואר שני במנהל עסקים. ...",False
1071,לא מבזבזים את הקול! לתיבות הדואר שלכם הגיעה ...,False


In [39]:
train_df, test_df = train_test_split(train_data, test_size=0.2, random_state = 10)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [40]:
# define hyperparameter
train_args ={"reprocess_input_data": True,
             "fp16":True,
             "num_train_epochs": 50,
             "overwrite_output_dir":True,
             "train_batch_size":32}

# Create a ClassificationModel
model = ClassificationModel(
     "bert", "bert-base-multilingual-cased",
     tokenizer_name="bert-base-multilingual-cased",
     num_labels=2,
     use_cuda=True,
     args=train_args
 )

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [41]:
model.train_model(train_df)

/home/politicalanalysis/environments/my_env/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/858 [00:00<?, ?it/s]

Epoch:   0%|          | 0/50 [00:00<?, ?it/s]

Running Epoch 0 of 50:   0%|          | 0/27 [00:00<?, ?it/s]

/home/politicalanalysis/environments/my_env/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


...

Running Epoch 46 of 50:   0%|          | 0/27 [00:00<?, ?it/s]

Running Epoch 47 of 50:   0%|          | 0/27 [00:00<?, ?it/s]

Running Epoch 48 of 50:   0%|          | 0/27 [00:00<?, ?it/s]

Running Epoch 49 of 50:   0%|          | 0/27 [00:00<?, ?it/s]

(1350, 0.07205702698033106)

In [44]:
def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')

result, model_outputs, wrong_predictions = model.eval_model(test_df,  acc=accuracy_score)

/home/politicalanalysis/environments/my_env/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/858 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/108 [00:00<?, ?it/s]

In [45]:
result

{'mcc': 0.9976703173155479,
 'tp': 439,
 'tn': 418,
 'fp': 1,
 'fn': 0,
 'auroc': 0.9999972817370788,
 'auprc': 0.9999948229447091,
 'acc': 0.9988344988344988,
 'eval_loss': 0.0016306187543604109}